In [ ]:

import numpy as np
import pandas as pd 
import os
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
%matplotlib inline
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/covid19s-impact-on-airport-traffic/covid_impact_on_airport_traffic.csv')

In [ ]:
df['long'] = df['Centroid'].apply(lambda x: x[6:-1].split(' ')[0])
df['lat'] = df['Centroid'].apply(lambda x: x[6:-1].split(' ')[1])

In [ ]:
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)

In [ ]:
for i in range(len(df)):
    lat = df.iloc[i]['lat']
    long = df.iloc[i]['long']
    radius=5
    folium.CircleMarker(location = [lat, long], radius=radius,
                        fill =True).add_to(marker_cluster)
    
world_map

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=df.groupby('City')['City'].agg('count').index,values=df.groupby('City')['City'].agg('count'), name="Airports by City"),
              1, 1)
fig.add_trace(go.Pie(labels=df.groupby('Country')['Country'].agg('count').index,values=df.groupby('Country')['Country'].agg('count'), name="Airports by Country"),
              1, 2)

fig.update_traces(hole=.4)

fig.update_layout(
    title_text="Airports by cities & countries",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Cities', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Countries', x=0.85, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
plt.figure(figsize=(20,20))
fig = sns.countplot(y='AirportName',data=df)
fig.axes.set_title("Airports",fontsize=50)
fig.tick_params(labelsize=20)
fig.set_xlabel("QTY of airports",fontsize=30)
fig.set_ylabel("Name of Airports",fontsize=30)

In [ ]:
fig = make_subplots(rows=2,cols=1)

t1 = go.Box(x=df['PercentOfBaseline'],name='Box Percent Of Baseline')
t2 = go.Histogram(x=df['PercentOfBaseline'],name='Histogram Percent Of Baseline')

fig.add_trace(t1,row=1,col=1)
fig.add_trace(t2,row=2,col=1)

fig.update_layout(height=600, width=800, title_text="Checking Baseline Percents")

fig.show()

In [ ]:
df['year'] = pd.to_datetime(df['Date']).dt.strftime('%d-%m-%Y')
df["month"] = pd.DatetimeIndex(df['year']).month
df['years'] = pd.DatetimeIndex(df['year']).year
df['day'] = pd.DatetimeIndex(df['year']).day

In [ ]:
import calendar
df['month_by_name'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df = df.sort_values(by="month")

In [ ]:
def Airport_month(Airport_name): #We can check specific airport Percent Of Baseline std and mean during reporting period
    data = df[df['AirportName']==Airport_name]
    rollstd = data['PercentOfBaseline'].rolling(window=15).std()
    rollmean = data['PercentOfBaseline'].rolling(window=15).mean()
    data1 = pd.concat([data['PercentOfBaseline'],data['year'],rollstd,rollmean],axis=1)
    data1.columns = ['PercentOfBaseline','Date','STD','Mean']
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['PercentOfBaseline'],
        name='PercentOfBaseline',
        marker_color='blue'
    ))
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['STD'],
        name='Standard Deviation of PercentOfBaseline',
        marker_color='Black'
    ))
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['Mean'],
        name='Mean Value of PercentOfBaseline',
        marker_color='green'
    ))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=90,title_text=Airport_name)
    fig.show()

In [ ]:
#For Exam
Airport_month('Kingsford Smith')

In [ ]:
for airport in df['AirportName'].unique(): #but here we can see whole airporta percent of baseline std and mean during reporting period
    data = df[df['AirportName']==airport]
    rollstd = data['PercentOfBaseline'].rolling(window=15).std()
    rollmean = data['PercentOfBaseline'].rolling(window=15).mean()
    data1 = pd.concat([data['PercentOfBaseline'],data['year'],rollstd,rollmean],axis=1)
    data1.columns = ['PercentOfBaseline','Date','STD','Mean']
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['PercentOfBaseline'],
        name='PercentOfBaseline',
        marker_color='blue'
    ))
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['STD'],
        name='Standard Deviation of PercentOfBaseline',
        marker_color='Black'
    ))
    fig.add_trace(go.Scatter(
        x=data1['Date'],
        y=data1['Mean'],
        name='Mean Value of PercentOfBaseline',
        marker_color='green'
    ))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=90,title_text=airport)
    fig.show()